In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt 

trainingdata = np.load("train.mat")
traindat = pd.DataFrame(data=trainingdata)
print(traindat)

label = []

for line in open("train.labels"):
    label.append(int(line))

labels = pd.DataFrame(data=label)
#print(labels[0:5])

            0         1         2         3         4         5         6    \
0     -0.076968 -0.004365  0.019731  0.009190 -0.050608  0.008721  0.038924   
1     -0.049463 -0.020196 -0.008273  0.017458 -0.000435  0.024264  0.063787   
2      0.010414 -0.018651  0.003080 -0.005116  0.032243  0.015521  0.030721   
3     -0.055945 -0.047393 -0.033989 -0.006274 -0.022510 -0.012531 -0.000599   
4     -0.073024  0.032800  0.062011 -0.028722  0.091838 -0.014950 -0.089704   
...         ...       ...       ...       ...       ...       ...       ...   
24995  0.003648  0.093710  0.041927 -0.005563 -0.023833 -0.002938  0.009442   
24996 -0.018505  0.007941  0.037602 -0.021426 -0.033456  0.011947  0.023734   
24997  0.013416  0.001527  0.011863 -0.006423 -0.032362 -0.018805  0.007866   
24998 -0.037110  0.023442  0.057732  0.020060  0.011861  0.004493  0.024458   
24999 -0.010869 -0.014892 -0.015014  0.014490  0.034367 -0.013109  0.014245   

            7         8         9    ...       240 

In [2]:
class Perceptron(object):
	def __init__(self, rate = 0.01, number_of_iterations = 100):
		self.rate = rate
		self.number_of_iterations = number_of_iterations

	def fit(self, X, y):
		""" Fit training data
		
		Parameters:
		------------
		X : array-like, shape = [number_of_samples, number_of_features]
			Training vectors.
		y : array-like, shape = [number_of_samples]
			Target values.

		Returns
		------------
		self : object

		"""
		
		self.weight_matrix = np.zeros(1 + X.shape[1])
		self.errors_list = []

		for _ in range(self.number_of_iterations):
			errors = 0
			for xi, target in zip(X, y):
				update = self.rate * (target - self.predict(xi))
				self.weight_matrix[1:] += update * xi
				self.weight_matrix[0] += update
				errors += int(update != 0.0)
			self.errors_list.append(errors)
		return self

	def dot_product(self, X):
		""" Calculate the dot product """
		return (np.dot(X, self.weight_matrix[1:]) + self.weight_matrix[0])

	def predict(self, X):
		""" Predicting the label for the input data """
		return np.where(self.dot_product(X) >= 0.0, 1, -1)

In [7]:
per = Perceptron()

per.fit(trainingdata, labels)

#print(per.weight_matrix)

testingdata = np.load("test.mat")
testdat = pd.DataFrame(data=testingdata)
print(testdat.shape)

results = []

for index, sample in testdat.iterrows():
    prediction = per.predict(sample)
    if prediction != 1:
        print("Testing Sample {} predicts: {}".format(index, prediction))

print("Done")

(25000, 250)
Done


In [14]:
import numpy as np
import scipy as sp
import pandas as pd

trainingdata = np.load("train.mat")
traindat = trainingdata[0:2500]#pd.DataFrame(data=trainingdata[0:1000])

label = []

for line in open("train.labels"):
    lbl = int(line)
    if lbl != 1:
        lbl = 0
    label.append(lbl)

labels = label[0:2500]#pd.DataFrame(data=label[0:1000])

def nonlin(x,deriv=False):
    if(deriv==True):
        return x*(1-x)
    #return sp.special.expit(x)
    return 1/(1+np.exp(-x))
    
def train_model(X, y):

    np.random.seed(1)

    # randomly initialize our weights with mean 0
    syn0 = 2*np.random.random(np.shape(X)) - 1
    syn1 = 2*np.random.random(np.shape(y)) - 1
    
    l2 = []
    
    for j in range(600):

        if (j % 100) == 0:
            print("Step {}".format(j))
        # Feed forward through layers 0, 1, and 2
        l0 = X
        l1 = nonlin(np.dot(l0,syn0.T))
        #print(l1.shape)
        #print(l0.T.shape)
        #print(syn0.shape)
        l2 = nonlin(np.dot(l1,syn1))

        # how much did we miss the target value?
        l2_error = y - l2

        if (j% 100) == 0:
            print("Error:" + str(np.mean(np.abs(l2_error))))

        # in what direction is the target value?
        # were we really sure? if so, don't change too much.
        l2_delta = l2_error*nonlin(l2,deriv=True)

        # how much did each l1 value contribute to the l2 error (according to the weights)?
        l1_error = l2_delta.dot(syn1.T)

        # in what direction is the target l1?
        # were we really sure? if so, don't change too much.
        l1_delta = l1_error * nonlin(l1,deriv=True)

        syn1 += l1.T.dot(l2_delta)
        syn0 += (l0.T.dot(l1_delta.T)).T
    
    return l2
        
theta = train_model(traindat, labels)
print(theta.shape)
print(theta)

Step 0
Error:0.498400000444744
Step 100
Error:0.4984000007389507
Step 200
Error:0.4984000021683491
Step 300
Error:0.5016
Step 400
Error:0.5016
Step 500
Error:0.5016
(2500,)
[1. 1. 1. ... 1. 1. 1.]
